<a href="https://colab.research.google.com/github/SamyukthaPremAnanth/admire-yourself/blob/main/training_and_testing_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
import tqdm
import glob
from sklearn.preprocessing import LabelBinarizer

In [ ]:
BASE_PATH="drive/MyDrive/data/UCF-101"
VIDEOS_PATH=os.path.join(BASE_PATH,"**","*.avi")
SEQUENCE_LENGTH=40

In [ ]:
def frame_generator():
  video_paths=tf.io.gfile.glob(VIDEOS_PATH)
  np.random.shuffle(video_paths)
  for video_path in video_paths:
    frames=[]
    cap=cv2.VideoCapture(video_path)
    num_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_every_frame=max(1,num_frames // SEQUENCE_LENGTH)
    current_frame=0

    label=os.path.basename(os.path.dirname(video_path))

    max_images=SEQUENCE_LENGTH
    while True:
      success , frame=cap.read()
      if not success:
        break

      if current_frame % sample_every_frame ==0:
        frame=frame[:,:,::-1]
        img=tf.image.resize(frame,(299,299))
        img=tf.keras.applications.inception_v3.preprocess_input(
            img)
        max_images -=1
        yield img,video_path
      
      if max_images==0:
        break
      current_frame +=1


dataset=tf.data.Dataset.from_generator(frame_generator,
           output_types=(tf.float32,tf.string),
           output_shapes=((299,299,3),()))
dataset=dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: ((None, 299, 299, 3), (None,)), types: (tf.float32, tf.string)>


In [ ]:
inception_v3=tf.keras.applications.InceptionV3(include_top=False, weights="imagenet")
X=inception_v3.output
pooling_output=tf.keras.layers.GlobalAveragePooling2D()(X)
feature_extraction_model=tf.keras.Model(inception_v3.input,pooling_output)


87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
current_path=None
all_features=[]

for img, batch_paths in tqdm.tqdm(dataset):
  batch_features=feature_extraction_model(img)
  batch_features=tf.reshape(batch_features,
                            (batch_features.shape[0],-1))
  for features , path in zip(batch_features.numpy(),batch_paths.numpy()):
    if path != current_path and current_path is not None:
      output_path=current_path.decode().replace('.avi','.npy')
      np.save( output_path,all_features)
      all_features=[]

    current_path=path
    all_features.append(features)


In [ ]:
import glob
LABLES=glob.glob("drive/MyDrive/data/train/*")
LABLES = [LABLES[i].split('/')[4] for i in range(len(LABLES))]
LABLES = sorted(LABLES)
print(len(LABLES))
encoder=LabelBinarizer()
encoder.fit(LABLES)

101


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=0.),
    tf.keras.layers.LSTM(512, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(LABLES),activation='softmax')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy' , 'top_k_categorical_accuracy'])

In [ ]:
test_file="drive/MyDrive/data/ucfTrainTestlist/testlist01.txt"
train_file="drive/MyDrive/data/ucfTrainTestlist/trainlist01.txt"
with open(test_file) as f:
  test_list=[row.strip() for row in list(f)]
with open(train_file) as f:
  train_list=[row.strip() for row in list(f)]
  train_list=[row.split(' ')[0] for row in train_list]
  
def make_generator(file_list):
  def generator():
    np.random.shuffle(file_list)
    for path in file_list:
      full_path=os.path.join(BASE_PATH,path).replace('.avi','.npy')

      label=os.path.basename(os.path.dirname(path))
      features=np.load(full_path)

      padded_sequence=np.zeros((SEQUENCE_LENGTH,2048))
      padded_sequence[0:len(features)]=np.array(features)

      transformed_label=encoder.transform([label])
      yield padded_sequence, transformed_label[0]
  return generator


In [ ]:
train_dataset = tf.data.Dataset.from_generator(make_generator(train_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 2048), 101))
train_dataset = train_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(make_generator(test_list),
                 output_types=(tf.float32, tf.int16),
                 output_shapes=((SEQUENCE_LENGTH, 2048), 101))
valid_dataset = valid_dataset.batch(16).prefetch(tf.data.experimental.AUTOTUNE)

print(train_dataset)
print(valid_dataset)

<PrefetchDataset shapes: ((None, 40, 2048), (None, 101)), types: (tf.float32, tf.int16)>
<PrefetchDataset shapes: ((None, 40, 2048), (None, 101)), types: (tf.float32, tf.int16)>


In [ ]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir='drive/MyDrive/data/models', update_freq=1000)
model.fit(train_dataset,epochs=15,callbacks=[tensorboard_callback],validation_data=valid_dataset)

Epoch 1/15
597/597 [==============================] - 3271s 5s/step - loss: 3.4913 - accuracy: 0.1896 - top_k_categorical_accuracy: 0.4019 - val_loss: 2.6189 - val_accuracy: 0.3249 - val_top_k_categorical_accuracy: 0.6606
Epoch 2/15
597/597 [==============================] - 318s 532ms/step - loss: 1.9509 - accuracy: 0.4688 - top_k_categorical_accuracy: 0.7650 - val_loss: 1.5665 - val_accuracy: 0.5752 - val_top_k_categorical_accuracy: 0.8372
Epoch 3/15
597/597 [==============================] - 318s 532ms/step - loss: 1.3883 - accuracy: 0.6022 - top_k_categorical_accuracy: 0.8708 - val_loss: 1.6973 - val_accuracy: 0.5432 - val_top_k_categorical_accuracy: 0.8091
Epoch 4/15
597/597 [==============================] - 291s 488ms/step - loss: 1.0744 - accuracy: 0.6881 - top_k_categorical_accuracy: 0.9180 - val_loss: 1.4011 - val_accuracy: 0.6130 - val_top_k_categorical_accuracy: 0.8810
Epoch 5/15
597/597 [==============================] - 282s 472ms/step - loss: 0.8963 - accuracy: 0.7318 - 

In [ ]:
path = 'drive/MyDrive/data/test'
model_file = os.path.join(path, 'my_model.hdf5')
model.save(model_file) 